In [101]:
import numpy as np
import math
from enum import Enum

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

k_b = 1.380649e-23
m_H = 1.67262192369e-27
m_NA = 3.81735881448E-26
m_e = 9.1093837015E-31
q_e = 1.602176634E-19

class p_type(Enum):
    Electron = 1
    Hydrogen = 2
    Natrium = 3

def runge_kutta4(v, B, h, c):
    k1 = c * np.cross(v, B)
    k2 = c * np.cross(v + 0.5 * h * k1, B)
    k3 = c * np.cross(v + 0.5 * h * k2, B)
    k4 = c * np.cross(v + h * k3, B)
    return h * (k1 / 6.0 + k2 / 3.0 + k3 / 3.0 + k4 / 6.0)

def run_sim(particle_type, T):
    if particle_type is p_type.Electron:
        q = -1.0 * q_e
        m = m_e
    elif particle_type is p_type.Hydrogen:
        q = q_e
        m = m_H
        t_step = 0.1
        T_end = 200.0
    elif particle_type is p_type.Natrium:
        q = 2.0 * q_e
        m = m_NA
        t_step = 2.0
        T_end = 1000.0
    N = int(T_end / t_step)
    q_per_m = q / m
    B_0 = 60e-9
    d = 0.195 * 2440000
    v0 = math.sqrt(3.0 * k_b * T / m)

    pos = np.array([0.0, 0.0, -100000.0])
    vel = np.array([0.0, v0, 0.0])

    y = np.zeros([N], dtype=float)
    z = np.zeros([N], dtype=float)

    for i in range(N):
        B = np.array([B_0 * math.tanh(pos[2] / d), 0.0, 0.0])
        dv = runge_kutta4(vel, B, t_step, q_per_m)
        vel = vel + dv
        pos = pos + t_step * vel
        y[i] = pos[1]
        z[i] = pos[2]
    return [y,z]

x, y = run_sim(p_type.Natrium, 18000)
p = figure(title="NSC sim", plot_height=300, plot_width=600, background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)
show(p, notebook_handle=True)

def update(particle_type = "Natrium", T=18000):
    if   particle_type == "Electron": pflag = p_type.Electron
    elif particle_type == "Hydrogen": pflag = p_type.Hydrogen
    elif particle_type == "Natrium": pflag = p_type.Natrium
    x, y = run_sim(pflag, T)
    r.data_source.data['x'] = x
    r.data_source.data['y'] = y
    push_notebook()

interact(update, particle_type=["Hydrogen", "Natrium"], T=(5000,50000,500))
#interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

Loading BokehJS ...

interactive(children=(Dropdown(description='particle_type', index=1, options=('Hydrogen', 'Natrium'), value='N…

<function __main__.update(particle_type='Natrium', T=18000)>